# Analyser les ventes d'un site de e-commerce spécialisé dans les livres

L'entreprise pour qui vous travaillez souhaite analyser les ventes de son site et mieux connaitre ses clients, ainsi que le détail des ventes de ses produits

Vous pourrez télécharger l'ensemble des données [via ce lien](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-analyst/dataset_P4.zip)  Les données que vous avez à disposition :   
* les ventes (appelées “Transactions”)  
* la liste des clients  
* la liste des produits.

## Corrélations des données

Vous allez réalisé l’analyse des données. Une grande liberté vous est laissée sur ce plan, mais gardez en tête la mission gloable : nous souhaitons comprendre les ventes.

Vous devrez y utiliser au moins :

* Y a-t-il une corrélation entre le sexe des clients et les catégories de produits achetés ?
* Y a-t-il une corrélation entre l'âge des clients et :  
    Le montant total des achats ;  
    La fréquence d’achat (ie. nombre d'achats par mois par exemple) ;  
    La taille du panier moyen (en nombre d’articles) ;  
    Les catégories de produits achetés.  

Il existe 3 corrélations possibles : 
* corrélation entre deux variables quantitatives : matrice de corrélation et heatmap sont souvent utilisés  
* corrélation entre une variable qualitative et une variable quantitative : [ANOVA](https://openclassrooms.com/fr/courses/4525266-decrivez-et-nettoyez-votre-jeu-de-donnees/4774896-analysez-une-variable-quantitative-et-une-qualitative-par-anova)  
* corrélation entre deux variables qualitatives : [Chi-2](https://openclassrooms.com/fr/courses/4525266-decrivez-et-nettoyez-votre-jeu-de-donnees/4775616-analysez-deux-variables-qualitatives-avec-le-chi-2)

In [224]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [225]:
transactions = pd.read_csv('datasets/transactions.csv')
customers = pd.read_csv('datasets/customers.csv')
products = pd.read_csv('datasets/products.csv')

In [226]:
transactions_copy = transactions.copy()
customers_copy = customers.copy()
products_copy = products.copy()

In [229]:
transactions = transactions.drop_duplicates()
transactions.duplicated().sum()

0

In [230]:
df_merge = customers_copy.merge(transactions_copy, on='client_id').merge(products_copy, on='id_prod')

In [231]:
df_merge

,client_id,sex,birth,id_prod,date,session_id,price,categ
0,c_4410,f,1967,0_1455,2021-03-22 14:29:25.189266,s_9942,8.99,0
1,c_4410,f,1967,0_1376,2021-09-24 22:58:27.418343,s_94984,16.24,0
2,c_4410,f,1967,1_312,2022-01-29 14:07:47.482092,s_156960,24.56,1
3,c_4410,f,1967,1_653,2021-07-29 23:34:41.866951,s_68860,25.99,1
4,c_4410,f,1967,0_1110,2021-11-04 16:28:30.169021,s_114715,4.71,0
...,...,...,...,...,...,...,...,...
336908,c_84,f,1982,1_459,2022-02-17 00:16:56.629536,s_166337,15.99,1
336909,c_84,f,1982,0_1050,2021-09-20 10:05:37.086910,s_92701,12.51,0
336910,c_84,f,1982,0_1417,2021-03-13 23:55:06.835238,s_5960,17.99,0
336911,c_84,f,1982,1_343,2021-04-13 03:59:33.614841,s_19845,27.99,1


## Nettoyage avant de commencer

In [249]:
transactions_copy.describe()

,date
count,336816
mean,2021-09-03 02:15:11.058547968
min,2021-03-01 00:01:07.843138
25%,2021-05-30 21:13:10.275369728
50%,2021-09-07 05:00:52.682486272
75%,2021-12-08 04:39:47.265316864
max,2022-02-28 23:59:58.040472


In [238]:
transactions_copy = transactions_copy[~transactions_copy['date'].astype(str).str.contains("test", case=False, na=False)]

In [239]:
transactions_copy['date'] = pd.to_datetime(transactions_copy['date'])

In [267]:
transactions_copy.dtypes

id_prod               object
date          datetime64[ns]
session_id            object
client_id             object
dtype: object

In [241]:
df_merge['sex'] = pd.get_dummies(df_merge['sex'], drop_first=True, dtype=int)

In [242]:
products_copy['price'].min()
products_copy['price'] = products_copy['price'].replace(-1,0) 

In [243]:
from datetime import datetime
date_now = datetime.now().year
date_now

2025

In [244]:
df_merge['age'] = (date_now - df_merge['birth']).astype(int)

In [245]:
df_merge.head()

,client_id,sex,birth,id_prod,date,session_id,price,categ,age
0,c_4410,0,1967,0_1455,2021-03-22 14:29:25.189266,s_9942,8.99,0,58
1,c_4410,0,1967,0_1376,2021-09-24 22:58:27.418343,s_94984,16.24,0,58
2,c_4410,0,1967,1_312,2022-01-29 14:07:47.482092,s_156960,24.56,1,58
3,c_4410,0,1967,1_653,2021-07-29 23:34:41.866951,s_68860,25.99,1,58
4,c_4410,0,1967,0_1110,2021-11-04 16:28:30.169021,s_114715,4.71,0,58


In [246]:
df_merge['age'].dtypes

dtype('int64')

Y a-t-il une corrélation entre l'âge des clients et :
Le montant total des achats ;
La fréquence d’achat (ie. nombre d'achats par mois par exemple) ;
La taille du panier moyen (en nombre d’articles) ;
Les catégories de produits achetés.

In [278]:
df_merge['purchase_sum'] = df_merge.groupby('client_id')['price'].transform('sum')
#afficher le montant d'achat par ans des clients
#group by client id / age

In [279]:
df_merge.head()

,client_id,sex,birth,id_prod,date,session_id,price,categ,age,purchase_sum
0,c_4410,0,1967,0_1455,2021-03-22 14:29:25.189266,s_9942,8.99,0,58,575.2
1,c_4410,0,1967,0_1376,2021-09-24 22:58:27.418343,s_94984,16.24,0,58,575.2
2,c_4410,0,1967,1_312,2022-01-29 14:07:47.482092,s_156960,24.56,1,58,575.2
3,c_4410,0,1967,1_653,2021-07-29 23:34:41.866951,s_68860,25.99,1,58,575.2
4,c_4410,0,1967,0_1110,2021-11-04 16:28:30.169021,s_114715,4.71,0,58,575.2
